In [ ]:
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import Checkbox, Box, Dropdown, fixed, interact, interactive, interact_manual, Label, Layout, Textarea
import json

# Override ipywidgets styles
display(HTML('''
<style>
textarea {min-height: 110px !important;}
/* Allow extra-long labels */
.widget-label {min-width: 25ex !important; font-weight: bold;}
.widget-checkbox {margin-left: -60px !important;}
/* Classes for toggling widget visibility */
.hidden {display: none;}
.visible {display: flex;}
</style>
'''))

class ConfigForm(object):
    # Define widgets
    caption_msg = '<h3>Create a Collection Manifest</h3>'
    caption = widgets.HTML(value=caption_msg)
    button = widgets.Button(description='Submit')
    action = widgets.Dropdown(options=['Create Manifest', 'Insert Manifest into Database'], value='Create Manifest')
    _id = widgets.Text(value='')
    path = widgets.Text(value='')
    publications = widgets.Textarea(value='', placeholder="List each publication on a separate line.")
    description = widgets.Textarea(value='')
    collectors = widgets.Textarea(value='', placeholder="List each collector on a separate line.")
    date = widgets.Textarea(placeholder='List each date on a separate line.\nDate ranges should use the separate start and end dates with commas.\nValid formats are YYYY-MM-DD and YYYY-MM-DDTHH:MM:SS.')
    workstation = widgets.Text(value='')
    query_terms = widgets.Textarea(value='', placeholder="List each query term on a separate line.")
    processes = widgets.Textarea(value='', placeholder="List each process on a separate line.")
    global_caption = widgets.HTML(value='<h4>Global Congfiguration</h4>')
    title = widgets.Text(value='')
    altTitle = widgets.Text(value='')
    label = widgets.Text(value='')
    precise = widgets.Checkbox(value=False)
    notes = widgets.Textarea(placeholder='List each note on a separate line.')

    
    # Configure widget layout
    flex = Layout(display='flex', flex_flow='row', justify_content='space-between')

    # Assemble widgets in Boxes
    form_items = [
        Box([Label(value='Action:'), action], layout=flex),
        Box([Label(value='_id (required):'), _id], layout=flex),
        Box([Label(value='path (required):'), path], layout=flex),
        Box([Label(value='publications (required):'), publications], layout=flex),
        Box([Label(value='description (required):'), description], layout=flex),
        Box([Label(value='collectors (required):'), collectors], layout=flex),
        Box([Label(value='date (required):'), date], layout=flex),
        Box([Label(value='workstation (optional):'), workstation], layout=flex),
        Box([Label(value='queryterms (optional):'), query_terms], layout=flex),
        Box([Label(value='processes (optional):'), processes], layout=flex),
        Box([Label(value='title (optional):'), title], layout=flex),
        Box([Label(value='altTitle (optional):'), altTitle], layout=flex),
        Box([Label(value='label (optional):'), label], layout=flex),
        Box([Label(value='notes (optional):'), notes], layout=flex)
    ]
    
    # Initialise the class object
    def __init__(self, object):
        self.caption = ConfigForm.caption
        self.button = ConfigForm.button
        self.action = ConfigForm.form_items[0]
        self._id = ConfigForm.form_items[1]
        self.path = ConfigForm.form_items[2]
        self.publications = ConfigForm.form_items[3]
        self.description = ConfigForm.form_items[4]
        self.collectors = ConfigForm.form_items[5]
        self.date = ConfigForm.form_items[6]
        self.workstation = ConfigForm.form_items[7]
        self.query_terms = ConfigForm.form_items[8]
        self.processes = ConfigForm.form_items[9]
        self.title = ConfigForm.form_items[10]
        self.altTitle = ConfigForm.form_items[11]
        self.label = ConfigForm.form_items[12]
        self.notes = ConfigForm.form_items[13]               

        # Ensure that a date is correctly formatted and start dates precede end dates
        def check_date_format(date):
            items = date.replace(' ', '').split(',')
            for item in items:
                if len(item) > 10:
                    format = '%Y-%m-%dT%H:%M:%S'
                else:
                    format = '%Y-%m-%d'
                try:
                    if item != datetime.strptime(item, format).strftime(format):
                        raise ValueError
                    return True
                except ValueError:
                    print('The date value ' + item + ' is in an incorrect format. Use YYYY-MM-DD or YYYY-MM-DDTHH:MM:SS.')
            if len(items) == 2:
                try:
                    assert items[1] > items[0]
                except:
                    print('Your end date ' + items[1] + ' must be after your start date ' + items[0] + '.')


        # Transform textarea with dates to a valid schema array
        def process_dates(datefield):
            date = datefield.strip().split('\n')
            new_date = []
            d = {}

            # Validate all the date formats individually
            for item in date:
                check_date_format(item)

            # Determine if the datelist is a mix of normal and precise dates
            contains_precise = []
            for item in date:
                if ',' in item:
                    start, end = item.replace(' ', '').split(',')
                    if len(start) > 10 or len(end) > 10:
                        contains_precise.append('precise')
                    else:
                        contains_precise.append('normal')
                elif len(item) > 10:
                    contains_precise.append('precise')
                else:
                    contains_precise.append('normal')
            # Handle a mix of normal and precise dates
            if 'normal' in contains_precise and 'precise' in contains_precise:
                d['normal'] = []
                d['precise'] = []
                for item in date:
                    if ',' in item:
                        start, end = item.replace(' ', '').split(',')
                        if len(start) > 10 or len(end) > 10:
                            d['precise'].append({'start': start, 'end': end})
                        else:
                            d['normal'].append({'start': start, 'end': end})
                    else:
                        if len(item) > 10:
                            d['precise'].append(item)
                        else:
                            d['normal'].append(item)
                new_date.append(d)
            # Handle only precise dates
            elif 'precise' in contains_precise:
                d['precise'] = []
                for item in date:
                    if ',' in item:
                        start, end = item.replace(' ', '').split(',')
                        d['precise'].append({'start': start, 'end': end})
                    else:
                        d['precise'].append(item)
                new_date.append(d)
            # Handle only normal dates
            else:
                for item in date:
                    if ',' in item:
                        start, end = item.replace(' ', '').split(',')
                        new_date.append({'start': start, 'end': end})
                    else:
                        new_date.append(item)

            return new_date

        
        def split_list(str):
            seq = str.split('\n')
            for key, value in enumerate(seq):
                seq[key] = value.strip()
            return seq
        
        def handle_submit(values):
            self.values = {}
            self.values['action'] = ConfigForm.action.value
            self.values['_id'] = ConfigForm._id.value.strip()
            self.values['path'] = ConfigForm.path.value.strip()
            self.values['publications'] = split_list(ConfigForm.publications.value)
            self.values['description'] = ConfigForm.description.value.strip()
            self.values['collectors'] = split_list(ConfigForm.collectors.value)
            self.values['date'] = process_dates(ConfigForm.date.value.strip())
            self.values['workstation'] = ConfigForm.workstation.value.strip()
            self.values['query_terms'] = split_list(ConfigForm.query_terms.value)
            self.values['processes'] = split_list(ConfigForm.processes.value)
            self.values['title'] = ConfigForm.title.value.strip()
            self.values['altTitle'] = ConfigForm.altTitle.value.strip()
            self.values['label'] = ConfigForm.label.value.strip()
            self.values['notes'] = split_list(ConfigForm.notes.value)
            required = ['_id', 'path', 'publications', 'description', 'collectors', 'date']
            error = False
            for item in required:
                try:
                    assert self.values[item]
                except:
                    error  = True
                    print('A value for ' + item + ' is required.')
            if error != True:
                manifest = json.dumps(self.values, indent=4)
                msg = 'You entered the following configuration. '
                if self.values['action'] == 'Create Manifest':
                    msg += 'To preview the manifest created from these options, '
                else:
                    msg += 'To insert the manifest based on these options, '
                msg += 'run the "Execute Action" cell below.\n\n'
                clear_output()
                print(msg)
                print(manifest)

        # Initialise widgets in the container Box
        self.form = Box([self.action, self._id, self.path, self.publications, self.description, 
                         self.collectors, self.date, self.workstation, self.query_terms, 
                         self.processes, self.title, self.altTitle, self.label, self.notes],
                   layout=Layout(
                    display='flex',
                    flex_flow='column',
                    border='solid 2px',
                    align_items='stretch',
                    width='50%'))

        # Modify the CSS and set up some helper variables
        box = self.form

        # Display the form and watch for changes
        display(self.caption)
        display(box)
        display(self.button)
        self.button.on_click(handle_submit)

# Instantiate the form - values accessible with e.g. config.values['delete_id]
config = ConfigForm(object)